In [30]:
from numpy.random import normal
import numpy as np
from numpy.linalg import inv

np.random.seed(0)

In [31]:
#Create the model
T = 10

#Model 1: linear model
#x_k+1 = 2*x_k + 0 + gaussian noise (mean 0,variance 0.5)
#y_k = x_k + gaussian noise  (mean 0,variance 0.5)
A = np.identity(1) * 2
B = np.zeros([1,1])
C = np.identity(1)
D = np.zeros([1,1])
Q = np.identity(1) * 5
R = np.identity(1) * 5
mean = np.zeros([1,1])

x = np.zeros([T,1])
y = np.zeros([T,1])
u = np.zeros([T,1])

print("Model 1: linear model"+ '\n' + \
"x_k+1 = "+ str(A.item()) +" * x_k + 0 + gaussian noise (mean " + str(mean.item()) +",variance "+ str(Q.item()) +")"+ '\n'+ \
"y_k = "+ str(C.item()) +" * x_k + gaussian noise  (mean "+ str(mean.item()) +",variance "+ str(R.item()) +")"+ '\n')

#Initialization
# for k in range(T-1):
#     u[k] = 0
x[0] = normal(0,1)

#Propagation
for k in range(T-1):
    x[k+1] = np.dot(A,x[k]) + np.dot(B,u[k]) + normal(mean,Q)
for k in range(T):
    y[k] = np.dot(C,x[k]) + np.dot(D,u[k]) + normal(mean,R)

print(x)
# print(y)

Model 1: linear model
x_k+1 = 2.0 * x_k + 0 + gaussian noise (mean 0.0,variance 5.0)
y_k = 1.0 * x_k + gaussian noise  (mean 0.0,variance 5.0)

[[  1.76405235e+00]
 [  5.52889073e+00]
 [  1.59514714e+01]
 [  4.31074088e+01]
 [  9.55526075e+01]
 [  1.86218826e+02]
 [  3.77188093e+02]
 [  7.53619400e+02]
 [  1.50672271e+03]
 [  3.01549841e+03]]


In [32]:
#Kalman filter
#from page 25 of ftp://icf.org.ru/pub/docs/linux-support/computer%20science/Artificial%20Intelligence/Neural%20networks/Kalman%20Filtering%20and%20Neural%20Networks%20-%20Simon%20Haykin.pdf

#Initialization
V = np.zeros([T,1])
V[0] = np.identity(1)
x_f = np.zeros([T,1])
x_f[0] = normal(0,V[0])
V_plus_all = np.zeros([T,1])
x_plus_all = np.zeros([T,1])

#Propagation
for k in range(1,T):
    x_plus = np.dot(A, x_f[k-1])
    x_plus_all[k] = x_plus
    V_plus = A.dot(V[k-1]).dot(A.T) + Q 
    V_plus_all[k] = V_plus
    
    K = V_plus.dot(C.T).dot(inv(C.dot(V_plus.dot(C.T) + R)))
    x_f[k] = x_plus + K.dot(y[k] - C.dot(x_plus))
    V[k] = (np.identity(K.shape[0]) - K.dot(C)).dot(V_plus)
    
print(x_f)
print('\n')
print(x_f - x)

[[ -2.55298982e+00]
 [  6.40517330e+00]
 [  1.82371540e+01]
 [  4.23028129e+01]
 [  9.52494087e+01]
 [  1.88386203e+02]
 [  3.83152279e+02]
 [  7.55212193e+02]
 [  1.50869605e+03]
 [  3.01240518e+03]]


[[-4.31704216]
 [ 0.87628257]
 [ 2.28568257]
 [-0.80459583]
 [-0.30319883]
 [ 2.16737762]
 [ 5.96418594]
 [ 1.59279279]
 [ 1.97334398]
 [-3.09322594]]


In [33]:
#Recursive
#F = A, B = H, P = V

#Initialization
x_b = np.zeros([T,1])
x_b[-1] = x_f[-1]
V_b = np.zeros([T,1])
V_b[-1] = V[-1]

#Propagation
for k in range(T-1, 0, -1): #from T-1 to 1 by -1 increments
    print(k)
    A_b = V[k-1].dot(A.T).dot(inv(np.matrix(V_plus_all[k])))
    V_b[k-1] = V[k-1] - A_b.dot(V_plus_all[k] - V_b[k]).dot(A_b.T)
    x_b[k-1] = x_f[k-1] + A_b.dot(x_b[k] - x_plus_all[k])
    
print(x_b)   
print('\n')
print("x_f - x_b = ",'\n',x_f - x_b)
print()
print("x_f - x = ",'\n',x_f - x)
print()
print("x_b - x = ",'\n',x_b - x)

9
8
7
6
5
4
3
2
1
[[  7.32252339e-01]
 [  9.67761006e+00]
 [  2.19004487e+01]
 [  4.61454061e+01]
 [  9.46778776e+01]
 [  1.89002265e+02]
 [  3.78385319e+02]
 [  7.53824447e+02]
 [  1.50679122e+03]
 [  3.01240518e+03]]


x_f - x_b =  
 [[-3.28524215]
 [-3.27243676]
 [-3.66329476]
 [-3.84259315]
 [ 0.57153104]
 [-0.61606173]
 [ 4.76696061]
 [ 1.38774676]
 [ 1.90483432]
 [ 0.        ]]

x_f - x =  
 [[-4.31704216]
 [ 0.87628257]
 [ 2.28568257]
 [-0.80459583]
 [-0.30319883]
 [ 2.16737762]
 [ 5.96418594]
 [ 1.59279279]
 [ 1.97334398]
 [-3.09322594]]

x_b - x =  
 [[-1.03180001]
 [ 4.14871933]
 [ 5.94897733]
 [ 3.03799732]
 [-0.87472987]
 [ 2.78343935]
 [ 1.19722533]
 [ 0.20504603]
 [ 0.06850966]
 [-3.09322594]]


We get 0, as Kalman filter does all the job on linear systems (we can use the backward recursion, but it helps only for non-linear systems)

# Now, take variable transition matrix (A_k and C_k, not A and C anymore)

In [36]:
#Create the model
T = 10
np.random.seed(0)
#Model 2: linear model with dependence over time
#x_k+1 = (k+1)*x_k + gaussian noise (mean 0,variance 1)
#y_k = (1 + 1/k)*x_k + gaussian noise  (mean 0,variance 0.1)
A = np.identity(1)
A_all = [A*k for k in range(1,T+1)]  #A_all is such that x[k+1] = np.dot(A_all[k],x[k]) + ...etc
B = np.zeros([1,1])
C_all = np.zeros([T,1]) #C_all is such that y[k] = np.dot(C_all[k],x[k]) + ...etc
for k in range(1,T):
    C = np.identity(1)
    C_all[k] = C*(1 + 1/(k+1))
D = np.zeros([1,1])
Q = np.identity(1) * 5
R = np.identity(1) * 5
mean = np.zeros([1,1])

x = np.zeros([T,1])
y = np.zeros([T,1])
u = np.zeros([T,1])

#Initialization
# for k in range(T-1):
#     u[k] = 0
x[0] = normal(0,1)

#Propagation
for k in range(T-1):
    x[k+1] = np.dot(A_all[k],x[k]) + np.dot(B,u[k]) + normal(mean,Q)
for k in range(T):
    y[k] = np.dot(C_all[k],x[k]) + np.dot(D,u[k]) + normal(mean,R)

print(x)
# print(y)

[[  1.76405235e+00]
 [  3.76483839e+00]
 [  1.24233667e+01]
 [  4.84745661e+01]
 [  2.03236054e+02]
 [  1.01129388e+03]
 [  6.07251373e+03]
 [  4.25068394e+04]
 [  3.40054199e+05]
 [  3.06048984e+06]]


In [37]:
#Kalman filter
#from page 25 of ftp://icf.org.ru/pub/docs/linux-support/computer%20science/Artificial%20Intelligence/Neural%20networks/Kalman%20Filtering%20and%20Neural%20Networks%20-%20Simon%20Haykin.pdf

#Initialization
V = np.zeros([T,1])
V[0] = np.identity(1)
x_f = np.zeros([T,1])
# x_f[0] = normal(0,V[0])
x_f[0] = 1.
V_plus_all = np.zeros([T,1])
x_plus_all = np.zeros([T,1])

#Propagation
for k in range(1,T):
    x_plus = np.dot(A_all[k-1], x_f[k-1])
    x_plus_all[k] = x_plus
    V_plus = A_all[k-1].dot(V[k-1]).dot(A_all[k-1].T) + Q 
    V_plus_all[k] = V_plus
    
    K = V_plus.dot(C_all[k].T).dot(inv(C_all[k].dot(np.matrix(V_plus.dot(C_all[k].T) + R))))
    x_f[k] = x_plus + K.dot(y[k] - C_all[k].dot(x_plus))
    V[k] = (np.identity(K.shape[0]) - K.dot(C_all[k])).dot(V_plus)
    
    print("x_plus",x_plus)
    print(C_all[k])
    print("h",C_all[k].dot(x_plus))
#     print("A_all",A_all)
#     print("C_all",C_all)
    print("V_plus",V_plus)
    print("K",K)
    print("x_f[k]",x_f[k])
    print("V[k]",V[k])
    print()
    
print(x_f)
print('\n')
print(x_f - x)

x_plus [ 1.]
[ 1.5]
h 1.5
V_plus [[ 6.]]
K [[ 0.42857143]]
x_f[k] [ 5.89369648]
V[k] [ 2.14285714]

x_plus [ 11.78739296]
[ 1.33333333]
h 15.7165239427
V_plus [[ 13.57142857]]
K [[ 0.58762887]]
x_f[k] [ 14.52172034]
V[k] [ 2.93814433]

x_plus [ 43.56516101]
[ 1.25]
h 54.456451263
V_plus [[ 31.44329897]]
K [[ 0.70971495]]
x_f[k] [ 48.35228163]
V[k] [ 3.54857475]

x_plus [ 193.4091265]
[ 1.2]
h 232.090951806
V_plus [[ 61.77719604]]
K [[ 0.78067912]]
x_f[k] [ 204.34771309]
V[k] [ 3.9033956]

x_plus [ 1021.73856547]
[ 1.16666667]
h 1192.02832638
V_plus [[ 102.58488999]]
K [[ 0.8227698]]
x_f[k] [ 1013.08541949]
V[k] [ 4.113849]

x_plus [ 6078.51251695]
[ 1.14285714]
h 6946.87144794
V_plus [[ 153.098564]]
K [[ 0.85069036]]
x_f[k] [ 6079.03538852]
V[k] [ 4.25345182]

x_plus [ 42553.24771965]
[ 1.125]
h 47872.4036846
V_plus [[ 213.41913921]]
K [[ 0.87075544]]
x_f[k] [ 42506.89287746]
V[k] [ 4.35377722]

x_plus [ 340055.14301972]
[ 1.11111111]
h 377839.0478
V_plus [[ 283.64174209]]
K [[ 0.88594

In [39]:
#Recursive
#F = A, B = H, P = V

#Initialization
x_b = np.zeros([T,1])
x_b[T-1] = x_f[T-1]
V_b = np.zeros([T,1])
V_b[T-1] = V[T-1]
# print(len(A_all))

#Propagation
for k in range(T-1, 0, -1): #from T-1 to 1 by -1 increments
    A_b = V[k-1].dot(A_all[k].T).dot(inv(np.matrix(V_plus_all[k])))
    V_b[k-1] = V[k-1] - A_b.dot(V_plus_all[k] - V_b[k]).dot(A_b.T)
    x_b[k-1] = x_f[k-1] + A_b.dot(x_b[k] - x_plus_all[k])
    
print(x_b)   
print('\n')
print("x_f - x_b = ",'\n',x_f - x_b)
print()
print("x_f - x = ",'\n',x_f - x)
print()
print("x_b - x = ",'\n',x_b - x)

[[  3.49239046e+00]
 [  8.47717137e+00]
 [  1.72413955e+01]
 [  5.08414849e+01]
 [  2.02076043e+02]
 [  1.01178830e+03]
 [  6.07161637e+03]
 [  4.25067161e+04]
 [  3.40053863e+05]
 [  3.06048614e+06]]


x_f - x_b =  
 [[-2.49239046]
 [-2.58347489]
 [-2.71967518]
 [-2.48920332]
 [ 2.27167031]
 [ 1.29712435]
 [ 7.41901588]
 [ 0.17677644]
 [ 1.7369051 ]
 [ 0.        ]]

x_f - x =  
 [[-0.76405235]
 [ 2.12885809]
 [ 2.09835364]
 [-0.12228446]
 [ 1.11165881]
 [ 1.79153745]
 [ 6.52165418]
 [ 0.0535231 ]
 [ 1.40155002]
 [-3.70402495]]

x_b - x =  
 [[ 1.72833811]
 [ 4.71233299]
 [ 4.81802882]
 [ 2.36691886]
 [-1.1600115 ]
 [ 0.4944131 ]
 [-0.89736171]
 [-0.12325334]
 [-0.33535509]
 [-3.70402495]]
